In [1]:
from New2048 import Game2048 as _2048
import numpy as np
import New2048

In [2]:
game = _2048(4)
game.start_game
game.playAdversaryAction()
#game.playRandUser()
print(game)
print(game.board_value())

+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |  2   |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
4


In [3]:
#New2048.main()

# TEST 1 - Monte Carlo Tree Search

In [4]:
# Starter Code: MCTS Implementation
import time


class Tree():
    def __init__(self, *, start_grid=None, parent=None, move=None):
        if parent is None:
            self.parent = None
            self.move = None
            self.grid = start_grid
        else:
            self.parent = parent
            self.move = move
            self.grid = parent.grid.copy()
            if self.grid.play_state:
                self.grid.playUserAction(move)
            else:
                self.grid.playAdversaryAction()
        
        self.values = []
        self.n = 0
        if self.is_terminal_state:
            self.unexplored_moves = set()
        else:
            if self.grid.play_state:
                self.unexplored_moves = set(self.grid.get_user_actions())
            else:
                #print(self.grid.getAdverseryActions())
                self.unexplored_moves = set(self.grid.getAdverseryActions())
        self.children = set([])
            
    @property
    def fully_expanded(self):
        return len(self.unexplored_moves) == 0
    
    @property
    def is_terminal_state(self):
        return (self.grid.winner is not None)


def monte_carlo_tree_search(start_grid, num_iterations=1000):
    """MCTS core loop"""
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout(leaf, start_grid)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move

def best_child(node):
    """When done sampling, pick the child visited the most."""
    #print(node)
    return max(node.children, key=lambda child: child.n)

'''def worst_child(node):
    """ Exact opposite of the best child (Least visited node) """
    return min(node.children, key=lambda child: child.n)'''


def best_uct(node, C=5):
    """Pick the best action according to the UCB/UCT algorithm"""
    chos_val = float("-inf")
    for nub in node.children:
        temp = np.sum(nub.values)/nub.n + C * np.sqrt(np.log(node.n)/nub.n)
        if temp >chos_val:
            chos_val = temp
            chose = nub
    try:
        return chose
    except:
        print(node.grid)
        return chose


def traverse(node):
    # If fully explored, pick one of the children
    while node.fully_expanded and not node.is_terminal_state:
        node = best_uct(node)

    # If the node is terminal, return it
    if node.is_terminal_state:
        return node
    
    # If the node is not terminal:
    # 1. pick a new move from 'unexplored_moves'
    move = node.unexplored_moves.pop()
    # 2. create a new child
    new_child = Tree(parent=node, move=move)
    # 3. add that child to the list of children
    node.children.add(new_child)
    # 4. return that new child
    return new_child


def rollout(node, start_grid):
    sample = node.grid.copy()
    winner = sample.play_random_moves_until_done()
    '''if winner == 0:
        return 0
    elif winner == start_grid.play_state:
        return 1
    else:
        return -1'''
    return sample.board_value()/(2048 ** 2)


def backpropagate(node, simulation_result):
    """Update the node and its parent (via recursion)."""
    if node is None:
        return
    node.n += 1
    node.values.append(simulation_result)
    backpropagate(node.parent, simulation_result)
    

In [18]:

time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(4)
    tot_time = 0
    while board.winner == None:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = monte_carlo_tree_search(board, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True
    if board.winner == 1:
        wins += 1
        win_arr.append(1)
    else:
        win_arr.append(0)
    time_arr.append(tot_time)
    print(i)
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)
    

0
+------+------+------+------+
|  2   |  32  |  2   |  4   |
+------+------+------+------+
|  16  | 1024 |  16  | 128  |
+------+------+------+------+
|  4   |  32  | 512  |  4   |
+------+------+------+------+
|  2   |  4   |  8   |  2   |
+------+------+------+------+
2
1329808.0
153214710464.22263
1
+------+------+------+------+
|  2   | 2048 |      | 1024 |
+------+------+------+------+
|  4   |  8   |  4   |      |
+------+------+------+------+
|  16  |  32  | 128  |  8   |
+------+------+------+------+
|  2   |  4   |  16  |  4   |
+------+------+------+------+
1
5261000.0
131083697658.1701
2
+------+------+------+------+
|  16  |  2   |  8   |  64  |
+------+------+------+------+
|  2   |  16  |  4   |  2   |
+------+------+------+------+
|  8   | 256  |  16  |  64  |
+------+------+------+------+
|  2   |  8   |  64  |  4   |
+------+------+------+------+
2
78832.0
197476740968.37006
3
+------+------+------+------+
|  8   |  4   |  2   |  4   |
+------+------+------+------+
| 

28
+------+------+------+------+
|  2   |  32  |  16  |  2   |
+------+------+------+------+
|  4   | 128  |  4   |  64  |
+------+------+------+------+
|  8   |  32  |  8   |  4   |
+------+------+------+------+
|  2   |  64  |  4   |  2   |
+------+------+------+------+
2
27088.0
163429064738.66992
29
+------+------+------+------+
|  8   |  4   |  2   |  8   |
+------+------+------+------+
|  2   | 128  |  4   |  2   |
+------+------+------+------+
|  4   |  32  | 256  |  16  |
+------+------+------+------+
|  2   |  4   |  8   |  2   |
+------+------+------+------+
2
83476.0
120869413342.80177
30
+------+------+------+------+
|  4   |  2   | 128  |  2   |
+------+------+------+------+
|  2   |  32  |  16  |  4   |
+------+------+------+------+
|  32  | 256  |  4   |  8   |
+------+------+------+------+
|  2   |  4   |  8   |  4   |
+------+------+------+------+
2
84448.0
170238611469.36172
31
+------+------+------+------+
|  64  |  2   |  32  |  2   |
+------+------+------+------+
|

56
+------+------+------+------+
|  4   |  8   |  64  |  4   |
+------+------+------+------+
|  32  | 128  |  16  |  8   |
+------+------+------+------+
|  4   |  16  |  8   |  64  |
+------+------+------+------+
|  2   |  8   |  2   |  8   |
+------+------+------+------+
2
26488.0
78309779746.80614
57
+------+------+------+------+
|  2   |  16  |  2   |  16  |
+------+------+------+------+
|  8   |  4   |  32  |  4   |
+------+------+------+------+
|  2   | 128  |  16  |  8   |
+------+------+------+------+
|  16  |  2   |  8   |  2   |
+------+------+------+------+
2
18676.0
59583528194.54465
58
+------+------+------+------+
|  8   |  4   |  2   |  4   |
+------+------+------+------+
|  4   | 512  |  16  |  8   |
+------+------+------+------+
|  8   |  16  |  64  |  4   |
+------+------+------+------+
|  2   |  8   | 128  |  2   |
+------+------+------+------+
2
283468.0
214500702790.9774
59
+------+------+------+------+
|  32  |  4   |  8   |  4   |
+------+------+------+------+
|  

84
+------+------+------+------+
|  4   |  16  |  4   |  2   |
+------+------+------+------+
| 128  |  4   |  16  |  4   |
+------+------+------+------+
|  64  |  2   | 128  |  16  |
+------+------+------+------+
|  2   |  32  |  4   |  2   |
+------+------+------+------+
2
38752.0
68095478081.47228
85
+------+------+------+------+
|  4   |  2   |  32  |  2   |
+------+------+------+------+
|  16  | 128  |  64  |  8   |
+------+------+------+------+
|  4   | 256  |  8   |  16  |
+------+------+------+------+
|  2   |  16  |  4   |  2   |
+------+------+------+------+
2
88000.0
143000504542.0773
86
+------+------+------+------+
|  4   |  2   |  16  |  2   |
+------+------+------+------+
|  16  |  8   | 128  |  8   |
+------+------+------+------+
|  2   |  64  |  8   |  4   |
+------+------+------+------+
|  4   |  32  |  4   |  2   |
+------+------+------+------+
2
22288.0
49369222037.75933
87
+------+------+------+------+
|  2   |  4   |  32  |  4   |
+------+------+------+------+
|  3

In [19]:
f = open("res.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 2 - n-Step MCTS

In [5]:

def mcts_with_n_step(start_grid, n_step, num_iterations = 1000):
    
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout_n_step(leaf, start_grid, n_step)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move


def rollout_n_step(node, start_grid, n_step):
    sample = node.grid.copy()
    winner = sample.play_random_moves_for_n(n_step)

    return sample.board_value()/(2048 ** 2)




In [ ]:
 time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(6)
    tot_time = 0
    while board.winner == None:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = mcts_with_n_step(board, 100, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True
    if board.winner == 1:
        wins += 1
        win_arr.append(1)
    else:
        win_arr.append(0)
    time_arr.append(tot_time)
    print(i)
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

0
+------+------+------+------+------+------+
|  4   | 1024 |  2   |  8   |  32  |  4   |
+------+------+------+------+------+------+
|      |  2   | 2048 |  2   | 128  |  32  |
+------+------+------+------+------+------+
|      |  4   |  8   | 256  |  8   |  16  |
+------+------+------+------+------+------+
|      |      | 512  |  4   |  2   |  4   |
+------+------+------+------+------+------+
|      |      |      |      |  2   |  4   |
+------+------+------+------+------+------+
|      |      |      |      |      |      |
+------+------+------+------+------+------+
1
5589556.0
137896996573.75082
1
+------+------+------+------+------+------+
|  16  |  2   | 512  |  2   | 2048 | 1024 |
+------+------+------+------+------+------+
|  2   |  4   | 1024 |  16  |  16  |  4   |
+------+------+------+------+------+------+
|  4   | 256  |  2   | 256  | 512  |  8   |
+------+------+------+------+------+------+
|      |  32  |  16  | 128  |  2   |  16  |
+------+------+------+------+------+-----

14
+------+------+------+------+------+------+
|      |      |      |  8   |  2   |  16  |
+------+------+------+------+------+------+
| 256  |  32  |  8   |  64  |  16  |  2   |
+------+------+------+------+------+------+
|  2   |  8   |  64  |  32  | 1024 |  4   |
+------+------+------+------+------+------+
|      |  4   | 128  |  4   |  8   |  2   |
+------+------+------+------+------+------+
|      |      | 512  |      | 512  | 2048 |
+------+------+------+------+------+------+
|      |      |      |      |  2   |  4   |
+------+------+------+------+------+------+
1
5860180.0
68097287077.16129
15
+------+------+------+------+------+------+
|  4   | 1024 |  2   |  8   |  2   | 128  |
+------+------+------+------+------+------+
|  2   |  16  | 128  |  32  | 256  |  2   |
+------+------+------+------+------+------+
|      |  2   |  64  |  4   |  8   |  32  |
+------+------+------+------+------+------+
|      |  2   |  4   |  8   | 256  |  8   |
+------+------+------+------+------+----

28
+------+------+------+------+------+------+
|  4   |  16  | 1024 |  16  |  2   |      |
+------+------+------+------+------+------+
|  4   |  64  |  8   |  32  |  8   |  4   |
+------+------+------+------+------+------+
|  8   |  4   |  32  |  4   |  32  |      |
+------+------+------+------+------+------+
|  4   |  8   |      |      |      |      |
+------+------+------+------+------+------+
| 2048 | 1024 |      | 1024 |      |      |
+------+------+------+------+------+------+
|  4   |      |  4   |      |  4   |      |
+------+------+------+------+------+------+
1
7348116.0
64692429535.7125
29
+------+------+------+------+------+------+
|      |      |      |  2   | 1024 | 2048 |
+------+------+------+------+------+------+
|      |      |      |      |  16  |  8   |
+------+------+------+------+------+------+
|      |  4   |  32  |  64  |  4   |  2   |
+------+------+------+------+------+------+
|      |      |      |  64  |  2   |  8   |
+------+------+------+------+------+-----

42
+------+------+------+------+------+------+
|  4   |  16  |  8   |  4   |      |      |
+------+------+------+------+------+------+
|  2   |  4   | 256  |  4   |  8   |      |
+------+------+------+------+------+------+
|  2   |  4   |  64  |  4   |  2   |      |
+------+------+------+------+------+------+
|  2   | 2048 |      |      |      |      |
+------+------+------+------+------+------+
|  4   |  8   |      |      |      |      |
+------+------+------+------+------+------+
|  8   |  32  |      |      |      |      |
+------+------+------+------+------+------+
1
4265600.0
80014329922.8656
43
+------+------+------+------+------+------+
|      |      |  2   |  8   |  16  |  32  |
+------+------+------+------+------+------+
|      |      |      |      | 2048 |  2   |
+------+------+------+------+------+------+
|      |  2   |  4   |  8   |  32  |  8   |
+------+------+------+------+------+------+
|      |      |      |  32  |  8   |  2   |
+------+------+------+------+------+-----

56
+------+------+------+------+------+------+
|      |      |      |      |      |      |
+------+------+------+------+------+------+
|  2   |  16  |      |      |      |      |
+------+------+------+------+------+------+
|  8   |  8   |  4   |      |      |      |
+------+------+------+------+------+------+
|  4   |  32  |  8   |  2   |  8   |  2   |
+------+------+------+------+------+------+
|  4   | 2048 |  16  |  4   |      |      |
+------+------+------+------+------+------+
|  16  |  2   |  8   |  32  |  8   |  2   |
+------+------+------+------+------+------+
1
4197588.0
69799742814.45586
57
+------+------+------+------+------+------+
|  2   | 512  |  8   |  4   | 2048 | 256  |
+------+------+------+------+------+------+
|  32  | 256  |  2   |  8   |  32  |  4   |
+------+------+------+------+------+------+
|  16  |  2   |  32  |  4   |  8   |      |
+------+------+------+------+------+------+
|  64  |  32  | 1024 |  16  |  4   |      |
+------+------+------+------+------+----

70
+------+------+------+------+------+------+
|      |  8   |  2   |  8   |  4   |  8   |
+------+------+------+------+------+------+
|      |      |  16  |  8   |  16  | 2048 |
+------+------+------+------+------+------+
|      |      |  8   |  32  |  2   |  8   |
+------+------+------+------+------+------+
|      |      |      |  64  | 128  | 2048 |
+------+------+------+------+------+------+
|      | 1024 |      | 1024 |  32  |  4   |
+------+------+------+------+------+------+
|      |      |      | 1024 |  2   | 1024 |
+------+------+------+------+------+------+
1
12606380.0
125980036204.67894
71
+------+------+------+------+------+------+
|  2   | 256  |  8   |  32  | 2048 |  16  |
+------+------+------+------+------+------+
|  16  | 512  |  16  |  8   | 1024 |  8   |
+------+------+------+------+------+------+
|  32  | 256  |  8   | 512  |      |      |
+------+------+------+------+------+------+
|  8   |  32  |      |  2   | 1024 |  8   |
+------+------+------+------+------+--

84
+------+------+------+------+------+------+
|  4   |  64  |  8   |  4   |  2   |      |
+------+------+------+------+------+------+
|  16  |  8   |  16  |  2   |  32  |      |
+------+------+------+------+------+------+
|  2   |  4   |  2   |  8   | 1024 |      |
+------+------+------+------+------+------+
| 256  | 128  | 256  |  4   |      |      |
+------+------+------+------+------+------+
|  2   |  4   | 2048 |      |      |      |
+------+------+------+------+------+------+
|  4   |  4   |  4   |      |      |      |
+------+------+------+------+------+------+
1
5396308.0
120872748642.88412
85
+------+------+------+------+------+------+
|      |      | 1024 |      | 1024 | 2048 |
+------+------+------+------+------+------+
|      |      |      |      |  2   |  16  |
+------+------+------+------+------+------+
|      |      |      |  16  | 512  |  8   |
+------+------+------+------+------+------+
|      |      |  4   | 512  |  64  |  4   |
+------+------+------+------+------+---

98
+------+------+------+------+------+------+
|  8   |  4   |  2   |  32  |  4   |  2   |
+------+------+------+------+------+------+
|  4   |  64  | 128  | 512  |  16  |  8   |
+------+------+------+------+------+------+
|      |      |  2   |  16  |  4   |  32  |
+------+------+------+------+------+------+
|      |      |      | 2048 | 512  |  2   |
+------+------+------+------+------+------+
|      |      |      |  2   |  4   |  2   |
+------+------+------+------+------+------+
|      |      |      |  2   |  4   |  4   |
+------+------+------+------+------+------+
1
4741900.0
202589559611.77078


In [ ]:
f = open("res_boot3.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 3 - DQ-learning

# TEST 4 - Adverseral 2048